<a href="https://colab.research.google.com/github/Wisdom-Nji/research.google-projects/blob/master/Word_Vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

In [8]:
import spacy

nlp = spacy.load("en_core_web_md")

In [35]:
class Category:
  URGENT = "URGENT" 
  RESULTS = "RESULT"
  QUESTIONS = "QUESTION"
  STATEMENTS = "STATEMENT"


In [47]:
# Data = { URGENT or RESULT}
train_x = ['The test was positive', 'Nobody got sick', 'We do not have TB', 'You results are available', "I'm not sick", "I am not negative"]
train_x1 = ['Please send help', 'I need someone right now', 'Her water just broke', "i've been shot"]
train_y = [Category.RESULTS, Category.RESULTS, Category.RESULTS, Category.RESULTS, Category.RESULTS, Category.RESULTS]
train_y1 = [Category.URGENT, Category.URGENT, Category.URGENT, Category.URGENT]

train_x += train_x1 
train_y += train_y1
print( len(train_y), ":", train_y )

10 : ['RESULT', 'RESULT', 'RESULT', 'RESULT', 'RESULT', 'RESULT', 'URGENT', 'URGENT', 'URGENT', 'URGENT']


In [67]:
# Data = {Question or Statement}
train_questions = ['What drug is this', 'would I have to pay', 'is this free', 'can this not be for free', 'should i come back later', 'am I tired']
train_label_statements = [Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS]
train_label_statements1 = [Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS, Category.STATEMENTS]
train_label_questions = [Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS, Category.QUESTIONS]

train_sq_data = train_label_statements + train_label_statements1 + train_label_questions

train_cat_data = []
train_cat_data += train_x + train_questions

In [65]:
# training cat data
docs = [nlp(text) for text in train_x]
train_x_word_vectors = [x.vector for x in docs]

In [66]:
# training sq data
docs = [nlp(text) for text in train_cat_data]
train_x_word_vectors_for_questions = [x.vector for x in docs]

In [70]:
from sklearn import svm
clf_svm_wv = svm.SVC(kernel='linear')
clf_svm_wv.fit(train_x_word_vectors, train_y)

# Training for Questions or statements
clf_svm_wv_sq = svm.SVC(kernel='linear')
clf_svm_wv_sq.fit(train_x_word_vectors_for_questions, train_sq_data)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [75]:
test_x = ["would you check for my drugs"]
test_docs = [nlp(text) for text in test_x]

test_x_word_vectors = [x.vector for x in test_docs]

print("Statement|Question:", clf_svm_wv_sq.predict( test_x_word_vectors ))
print("Urgent|Results:", clf_svm_wv.predict( test_x_word_vectors ))

Statement|Question: ['QUESTION']
Urgent|Results: ['RESULT']
